In [1]:
#This code uses a best feature select of 12 and calculates the results for 
#poly equal to 2. The result that are initially saved are for poly = 3 but 
#reruning the code should produce results for poly = 2

from sklearn.datasets import fetch_openml
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn.linear_model as skl_lm
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score

from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures

from sklearn import utils
from sklearn import preprocessing
from sklearn import neighbors
from pandas import read_csv

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn import preprocessing

from sklearn.feature_selection import SelectKBest

import csv
lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()
kfld = KFold(n_splits=3,random_state=42)

In [3]:
#Reads data in from data folder.


data = read_csv('data/ethylene_methane.txt',delim_whitespace=True)
print(data.shape)

(4178504, 19)


In [4]:
#creates a new attribute thats the % Methane, in cases where we divide by zero
#we recieve a NAN which is replaced by a 0
#the 4th line drops that attributes that we defined to be unnecasry from our K
#best select we ran in a previous part of our code
#this code also finds all unique PPM for methane

data['Methane%'] = (data['Methane(ppm)']/(data['Methane(ppm)']+data['Ethylene(ppm)']))
data['Methane%'] = data['Methane%'].fillna(0)

ppm = data['Methane(ppm)'].unique()

data = data.drop(columns = ['Time(sec)', 'sensor1', 'sensor2','sensor9','sensor10'])

sensor_name = ['sensor3', 'sensor4', 'sensor5','sensor6', 'sensor7', 'sensor8', 'sensor11','sensor12', 'sensor13', 'sensor14', 'sensor15', 'sensor16']


In [5]:
#This block searches for Outliers for each PPM of Methane and then drops them
#from the file.


for j in sensor_name:
    print(j)
    for i in np.sort(ppm):    
        data2 = data[data["Methane(ppm)"] == i] #only select a specific PPM to begin trimming
        
        results = data2[j].quantile([0.125,0.875]) #Next lines calculate the range for non-Outliers
        IQR_15 = 1.5*(results.loc[0.875] - results.loc[0.125])
        fq = results.loc[0.125] - IQR_15
        tq = results.loc[0.875] + IQR_15
                
        index = data2[(fq>data2[j]) | (tq<data2[j])] #Finds Outliers
        data = data.drop(index.index) #Uses index of Outliers to drop rows from data 


sensor3
sensor4
sensor5
sensor6
sensor7
sensor8
sensor11
sensor12
sensor13
sensor14
sensor15
sensor16


In [6]:
#this line adds a addition attribute that is the sensor value raised to some power
#by adding an additional number or change the two you can increase the power of the attribute
for i in [2]:
    for k in sensor_name:
        data[k+'**%i' %i]=data[k]**i
data = data.dropna()
data.head(2)

,Methane(ppm),Ethylene(ppm),sensor3,sensor4,sensor5,sensor6,sensor7,sensor8,sensor11,sensor12,...,sensor5**2,sensor6**2,sensor7**2,sensor8**2,sensor11**2,sensor12**2,sensor13**2,sensor14**2,sensor15**2,sensor16**2
3549,0.0,0.0,3436.85,3786.60,2163.69,2749.00,2708.66,2928.48,2681.90,2537.24,...,4.681554e+06,7.557001e+06,7.336839e+06,8.575995e+06,7.192588e+06,6.437587e+06,4.146110e+06,5.153309e+06,5.048335e+06,3.543204e+06
3554,0.0,0.0,3439.20,3773.93,2156.47,2719.97,2707.34,2920.05,2665.17,2515.02,...,4.650363e+06,7.398237e+06,7.329690e+06,8.526692e+06,7.103131e+06,6.325326e+06,4.125977e+06,5.157759e+06,5.086829e+06,3.593603e+06


In [7]:
#Defines our X and y and sets y equal to 1 if the Methane % is greater than 50%
#and splites the data into train and test

X_best = data.drop(columns = ['Methane(ppm)','Ethylene(ppm)','Methane%'])
y = np.where(data['Methane%'] >= 0.5, 1, 0)

start =time.time()
X_train, X_test, y_train, y_test = train_test_split(X_best, y, test_size=0.90, random_state=42) 
end = time.time()
print('Time (sec) = ',(end-start))

Time (sec) =  1.2000904083251953


In [8]:
#these next blocks of code are the model fitting, each block attempts to fit 
#one model and records the start time starting at the fitting and ends after 
#finishing the CV
#The score and time printed once the code finishes

start =time.time()
clf = LogisticRegression(random_state=42, solver='lbfgs',max_iter=10000,n_jobs = -1).fit(X_train, y_train)
score = cross_val_score(clf, X_test, y_test, cv=kfld, scoring='r2').mean()
end = time.time()
print(score)
print('Time (sec) = ',(end-start))
#2 R2 = 0.9203444712971587 Time (sec) =  1865.4750816822052
#3 R2

0.9203444712971587
Time (sec) =  2112.9818210601807


In [9]:
start =time.time()
lda.fit(X_train, y_train)
score = cross_val_score(lda, X_test, y_test, cv=kfld, scoring='r2').mean()
end = time.time()
print(score)
print('Time (sec) = ',(end-start))
#2 R2 = 0.9098351752134888 Time (sec) =  28.132073640823364
#3 R2

0.9098351752134888
Time (sec) =  34.80310606956482


In [10]:
start =time.time()
qda.fit(X_train, y_train)
score = cross_val_score(qda, X_test, y_test, cv=kfld, scoring='r2').mean()
end = time.time()
print(score)
print('Time (sec) = ',(end-start))
#2 r2 = 0.8771708325881379 Time (sec) =  12.298192024230957
#3 R2

0.8771708325881379
Time (sec) =  19.045295000076294


In [11]:
start =time.time()
knn = neighbors.KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
score = cross_val_score(knn, X_test, y_test, cv=kfld, scoring='r2').mean()
end = time.time()
print(score)
print('Time (sec) = ',(end-start))
#2 R2 = 0.966541590213955 Time (sec) =  808.5260109901428
#3 R2

0.966541590213955
Time (sec) =  797.9341518878937


In [13]:
#these next lines take the previously fitted models and computes
#recall and precision

clf_y = clf.predict(X_test)
print(recall_score(y_test, clf_y))
print(precision_score(y_test, clf_y))

0.9571519481358848
0.9994296130128697


In [14]:
lda_y = lda.predict(X_test)
print(recall_score(y_test, lda_y))
print(precision_score(y_test, lda_y))

0.9550334093993765
0.9957178400653112


In [15]:
qda_y = qda.predict(X_test)
print(recall_score(y_test, qda_y))
print(precision_score(y_test, qda_y))

0.9563639437011847
0.9763542535062575


In [16]:
knn_y = knn.predict(X_test)
print(recall_score(y_test, knn_y))
print(precision_score(y_test, knn_y))

0.9867549380782635
0.9925779276959007
